In [10]:
import pandas as pd
import csv
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

In [11]:
data = pd.read_csv("Root_Insurance_data.csv",delimiter = ",")

In [12]:
## Input: Data Frame (in the format of "Root_Insurance_data.csv") and encode_rank (default = False)
## Output: Data Frame (processed by one hot encoding ready to be used to make models)

## This function will one hot encode the following columns in the data frame: 
## Currently Insured, Marital Status, click, rank (optional, determined by encode_rank = True)
## and will drop the respective columns associated to the respective values of above columns:
## unknown, S, False, 5.0 (if encode_rank = True)

import category_encoders as ce

def prepare_data(data:pd.DataFrame(), encode_rank = False, drop_column = False) -> pd.DataFrame():
    temp = data.copy()
    
    if encode_rank:
        one_hot_cols = ["Currently Insured", "Marital Status", "click", "rank"]
        drop_cols = ["Currently Insured_unknown", "Marital Status_S", "rank_5.0", "click_False"]
        column_names = ['Currently Insured_Y', 'Currently Insured_N', 'Number of Vehicles',
                        'Number of Drivers', 'Marital Status_M', 'bid', 'rank_1.0',
                        'rank_2.0', 'rank_3.0', 'rank_4.0', 'click_True', 'policies_sold']
    else:
        one_hot_cols = ["Currently Insured", "Marital Status", "click"]
        drop_cols = ["Currently Insured_unknown", "Marital Status_S", "click_False"]
        column_names = ['Currently Insured_Y', 'Currently Insured_N', 'Number of Vehicles','Number of Drivers',
                        'Marital Status_M', 'bid', 'rank', 'click_True', 'policies_sold']
    
    encoder = ce.OneHotEncoder(cols = one_hot_cols, use_cat_names = True)
    temp = encoder.fit_transform(temp)
    temp = temp.drop(columns = drop_cols)
    temp = temp.reindex(columns=column_names)
    return temp

In [13]:
df = prepare_data(data)

C:\Users\David\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
from sklearn.metrics import average_precision_score, roc_auc_score, precision_recall_curve, accuracy_score

features = ['Currently Insured_Y',
            'Currently Insured_N',
            'Number of Vehicles',
            'Number of Drivers',
            'Marital Status_M',
            'bid',
            'rank']
predictor_var = "click_True"

df_train, df_test = train_test_split(df,
                                     shuffle=True,
                                     random_state=12345,
                                     test_size=.2,
                                     stratify=df[predictor_var])

In [15]:
## Allows for variantion of resampling with condition resampling = "None", "Up", "Down" or "Balanced"

import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
import keras.backend as K
from sklearn.utils import resample,shuffle
from keras.utils.np_utils import to_categorical

def average_precision(y_true, y_pred):
    return K.sum(average_precision_score(y_true.numpy(), y_pred.numpy()))

def obtain_nn_scores(data:pd.DataFrame(), num_of_splits = 5, resampling = "None") -> []:
    
    avg_precision_cross_val = np.zeros(num_of_splits)
    roc_auc_cross_val = np.zeros(num_of_splits)
    accuracy_cross_val = np.zeros(num_of_splits)
    

    kfold = StratifiedKFold(n_splits=num_of_splits, shuffle=True, random_state=77)
    
    i = 0
    for train_index, test_index in kfold.split(df_train[features], df_train[predictor_var]):
        df_train_train = resample_data(df.iloc[train_index], resample_type = "None")
        df_train_val = df.iloc[test_index]

        model = models.Sequential()
        model.add(layers.Dense(len(features), activation='relu', input_dim= len(features)))
        model.add(layers.Dense(len(features), activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop', 
                      loss='binary_crossentropy',
                      metrics=[average_precision, 
                               keras.metrics.AUC(name = 'AUCROC'),
                               keras.metrics.BinaryAccuracy()],
                      run_eagerly=True)
        history = model.fit(df_train_train[features], 
                        df_train_train[predictor_var],
                        epochs = 200,
                        batch_size = 1024,
                        validation_data=(df_train_val[features], 
                                         df_train_val[predictor_var]))

        y_proba = model.predict(df_test[features])
        y_pred = [int(x > .5) for x in y_proba]
        
        avg_precision_cross_val[i] = average_precision_score(df_test[predictor_var], y_proba)
        roc_auc_cross_val[i] = roc_auc_score(df_test[predictor_var], y_proba)
        accuracy_cross_val[i] = accuracy_score(df_test[predictor_var], y_pred)
        i += 1
    
    return [avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val]

def resample_data(df:pd.DataFrame(), resample_type = "None") -> pd.DataFrame():
    df_minority = df[df[predictor_var] == 1]
    df_majority = df[df[predictor_var] != 1]
    if resample_type == "Up":
        df_minority_up = resample(df_minority,
                                  random_state=886,
                                  n_samples=len(df_majority),
                                  replace=True)
        df_resampled = pd.concat([df_minority_up, df_majority]).sample(frac=1, random_state = 0)
    elif resample_type == "Down":
        df_majority_down = resample(df_majority,
                                    random_state=886,
                                    n_samples=len(df_minority),
                                    replace=True)
        df_resampled = pd.concat([df_majority_down, df_minority]).sample(frac=1, random_state = 0)
    elif resample_type == "Balanced":
        df_majority_bal = resample(df_majority,
                                           random_state=886,
                                           n_samples= int(len(df)/2),
                                           replace=True)
        df_minority_bal = resample(df_minority,
                                               random_state=886,
                                               n_samples=int(len(df)/2),
                                               replace=True)
        df_resampled = pd.concat([df_majority_bal, df_minority_bal]).sample(frac=1, random_state = 0) 
    else:
        df_resampled = df
    return df_resampled

In [16]:
def info(avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val):
    print("Average cross val AUCPR = " + str(np.average(avg_precision_cross_val)))
    print()
    print("Average cross val AUCROC = " + str(np.average(roc_auc_cross_val)))
    print()
    print("Average cross val Accuracy = " + str(np.average(accuracy_cross_val)))

### No Resample Case

In [17]:
avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val = obtain_nn_scores(df_train)
print()
info(avg_precision_cross_val, roc_auc_cross_val, accuracy_cross_val)

Epoch 1/200
7/7 [==============================] - 0s 35ms/step - loss: 0.9657 - average_precision: 0.1150 - AUCROC: 0.3410 - binary_accuracy: 0.8197 - val_loss: 0.9608 - val_average_precision: 0.1259 - val_AUCROC: 0.2688 - val_binary_accuracy: 0.7987
Epoch 2/200
7/7 [==============================] - 0s 27ms/step - loss: 0.8243 - average_precision: 0.1116 - AUCROC: 0.2529 - binary_accuracy: 0.8197 - val_loss: 0.8512 - val_average_precision: 0.1250 - val_AUCROC: 0.2382 - val_binary_accuracy: 0.7987
Epoch 3/200
7/7 [==============================] - 0s 27ms/step - loss: 0.7383 - average_precision: 0.1138 - AUCROC: 0.2318 - binary_accuracy: 0.8197 - val_loss: 0.7661 - val_average_precision: 0.1241 - val_AUCROC: 0.2317 - val_binary_accuracy: 0.7987
Epoch 4/200
7/7 [==============================] - 0s 26ms/step - loss: 0.6718 - average_precision: 0.1085 - AUCROC: 0.2242 - binary_accuracy: 0.8197 - val_loss: 0.7048 - val_average_precision: 0.1239 - val_AUCROC: 0.2273 - val_binary_accuracy:

7/7 [==============================] - 0s 27ms/step - loss: 0.3846 - average_precision: 0.4307 - AUCROC: 0.7950 - binary_accuracy: 0.8214 - val_loss: 0.4134 - val_average_precision: 0.4637 - val_AUCROC: 0.7898 - val_binary_accuracy: 0.8019
Epoch 34/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3842 - average_precision: 0.4266 - AUCROC: 0.7953 - binary_accuracy: 0.8214 - val_loss: 0.4145 - val_average_precision: 0.4671 - val_AUCROC: 0.7909 - val_binary_accuracy: 0.7987
Epoch 35/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3840 - average_precision: 0.4431 - AUCROC: 0.7965 - binary_accuracy: 0.8231 - val_loss: 0.4109 - val_average_precision: 0.4767 - val_AUCROC: 0.7912 - val_binary_accuracy: 0.8125ss: 0.3840 - average_precision: 0.4431 - AUCROC: 0.7965 - binary_accuracy: 0.82
Epoch 36/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3838 - average_precision: 0.4540 - AUCROC: 0.7957 - binary_accuracy: 0.8236 - val_loss: 0.4111 - val

7/7 [==============================] - 0s 24ms/step - loss: 0.3823 - average_precision: 0.4378 - AUCROC: 0.7974 - binary_accuracy: 0.8188 - val_loss: 0.4105 - val_average_precision: 0.4792 - val_AUCROC: 0.7906 - val_binary_accuracy: 0.8169
Epoch 66/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3818 - average_precision: 0.4403 - AUCROC: 0.7985 - binary_accuracy: 0.8255 - val_loss: 0.4127 - val_average_precision: 0.4775 - val_AUCROC: 0.7909 - val_binary_accuracy: 0.8100
Epoch 67/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3820 - average_precision: 0.4357 - AUCROC: 0.7983 - binary_accuracy: 0.8234 - val_loss: 0.4136 - val_average_precision: 0.4756 - val_AUCROC: 0.7915 - val_binary_accuracy: 0.8100
Epoch 68/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3818 - average_precision: 0.4375 - AUCROC: 0.7989 - binary_accuracy: 0.8242 - val_loss: 0.4104 - val_average_precision: 0.4791 - val_AUCROC: 0.7910 - val_binary_accuracy: 0.8119
E

7/7 [==============================] - 0s 30ms/step - loss: 0.3817 - average_precision: 0.4577 - AUCROC: 0.7992 - binary_accuracy: 0.8234 - val_loss: 0.4100 - val_average_precision: 0.4781 - val_AUCROC: 0.7908 - val_binary_accuracy: 0.8069
Epoch 98/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3814 - average_precision: 0.4551 - AUCROC: 0.7988 - binary_accuracy: 0.8250 - val_loss: 0.4100 - val_average_precision: 0.4791 - val_AUCROC: 0.7903 - val_binary_accuracy: 0.8069
Epoch 99/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3818 - average_precision: 0.4535 - AUCROC: 0.7984 - binary_accuracy: 0.8239 - val_loss: 0.4101 - val_average_precision: 0.4790 - val_AUCROC: 0.7907 - val_binary_accuracy: 0.8069
Epoch 100/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3815 - average_precision: 0.4437 - AUCROC: 0.7988 - binary_accuracy: 0.8239 - val_loss: 0.4154 - val_average_precision: 0.4669 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8100


7/7 [==============================] - 0s 27ms/step - loss: 0.3816 - average_precision: 0.4446 - AUCROC: 0.7989 - binary_accuracy: 0.8238 - val_loss: 0.4114 - val_average_precision: 0.4732 - val_AUCROC: 0.7900 - val_binary_accuracy: 0.8112
Epoch 130/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3815 - average_precision: 0.4441 - AUCROC: 0.7986 - binary_accuracy: 0.8266 - val_loss: 0.4137 - val_average_precision: 0.4653 - val_AUCROC: 0.7901 - val_binary_accuracy: 0.8025
Epoch 131/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3815 - average_precision: 0.4478 - AUCROC: 0.7985 - binary_accuracy: 0.8250 - val_loss: 0.4107 - val_average_precision: 0.4765 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.8131
Epoch 132/200
7/7 [==============================] - 0s 29ms/step - loss: 0.3821 - average_precision: 0.4448 - AUCROC: 0.7978 - binary_accuracy: 0.8231 - val_loss: 0.4104 - val_average_precision: 0.4772 - val_AUCROC: 0.7908 - val_binary_accuracy: 0.815

...

7/7 [==============================] - 0s 25ms/step - loss: 0.3911 - average_precision: 0.4512 - AUCROC: 0.7960 - binary_accuracy: 0.8186 - val_loss: 0.3847 - val_average_precision: 0.4167 - val_AUCROC: 0.7898 - val_binary_accuracy: 0.8138
Epoch 66/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3909 - average_precision: 0.4665 - AUCROC: 0.7961 - binary_accuracy: 0.8189 - val_loss: 0.3843 - val_average_precision: 0.4165 - val_AUCROC: 0.7896 - val_binary_accuracy: 0.8156
Epoch 67/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3908 - average_precision: 0.4555 - AUCROC: 0.7960 - binary_accuracy: 0.8194 - val_loss: 0.3843 - val_average_precision: 0.4165 - val_AUCROC: 0.7897 - val_binary_accuracy: 0.8138
Epoch 68/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3908 - average_precision: 0.4468 - AUCROC: 0.7961 - binary_accuracy: 0.8200 - val_loss: 0.3839 - val_average_precision: 0.4167 - val_AUCROC: 0.7892 - val_binary_accuracy: 0.8175
E

7/7 [==============================] - 0s 27ms/step - loss: 0.3886 - average_precision: 0.4692 - AUCROC: 0.7996 - binary_accuracy: 0.8231 - val_loss: 0.3827 - val_average_precision: 0.4161 - val_AUCROC: 0.7904 - val_binary_accuracy: 0.8188
Epoch 98/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3886 - average_precision: 0.4787 - AUCROC: 0.7989 - binary_accuracy: 0.8228 - val_loss: 0.3836 - val_average_precision: 0.4130 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8138
Epoch 99/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3890 - average_precision: 0.4720 - AUCROC: 0.7983 - binary_accuracy: 0.8253 - val_loss: 0.3831 - val_average_precision: 0.4143 - val_AUCROC: 0.7896 - val_binary_accuracy: 0.8150
Epoch 100/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3886 - average_precision: 0.4668 - AUCROC: 0.7991 - binary_accuracy: 0.8252 - val_loss: 0.3829 - val_average_precision: 0.4144 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.8163


7/7 [==============================] - 0s 24ms/step - loss: 0.3882 - average_precision: 0.4791 - AUCROC: 0.7996 - binary_accuracy: 0.8253 - val_loss: 0.3831 - val_average_precision: 0.4150 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8169
Epoch 130/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3882 - average_precision: 0.4796 - AUCROC: 0.7993 - binary_accuracy: 0.8253 - val_loss: 0.3844 - val_average_precision: 0.4140 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8169
Epoch 131/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3887 - average_precision: 0.4704 - AUCROC: 0.7997 - binary_accuracy: 0.8258 - val_loss: 0.3830 - val_average_precision: 0.4180 - val_AUCROC: 0.7903 - val_binary_accuracy: 0.8169
Epoch 132/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3884 - average_precision: 0.4775 - AUCROC: 0.7994 - binary_accuracy: 0.8219 - val_loss: 0.3830 - val_average_precision: 0.4180 - val_AUCROC: 0.7897 - val_binary_accuracy: 0.816

7/7 [==============================] - 0s 28ms/step - loss: 0.3881 - average_precision: 0.4815 - AUCROC: 0.7997 - binary_accuracy: 0.8211 - val_loss: 0.3830 - val_average_precision: 0.4162 - val_AUCROC: 0.7899 - val_binary_accuracy: 0.8169
Epoch 162/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3883 - average_precision: 0.4644 - AUCROC: 0.7987 - binary_accuracy: 0.8247 - val_loss: 0.3832 - val_average_precision: 0.4170 - val_AUCROC: 0.7903 - val_binary_accuracy: 0.8188
Epoch 163/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3882 - average_precision: 0.4614 - AUCROC: 0.7995 - binary_accuracy: 0.8216 - val_loss: 0.3831 - val_average_precision: 0.4134 - val_AUCROC: 0.7897 - val_binary_accuracy: 0.8169
Epoch 164/200
7/7 [==============================] - 0s 30ms/step - loss: 0.3884 - average_precision: 0.4597 - AUCROC: 0.7992 - binary_accuracy: 0.8242 - val_loss: 0.3830 - val_average_precision: 0.4162 - val_AUCROC: 0.7907 - val_binary_accuracy: 0.816

7/7 [==============================] - 0s 25ms/step - loss: 0.3881 - average_precision: 0.4694 - AUCROC: 0.7999 - binary_accuracy: 0.8244 - val_loss: 0.3832 - val_average_precision: 0.4166 - val_AUCROC: 0.7900 - val_binary_accuracy: 0.8188
Epoch 194/200
7/7 [==============================] - 0s 33ms/step - loss: 0.3882 - average_precision: 0.4768 - AUCROC: 0.7996 - binary_accuracy: 0.8234 - val_loss: 0.3833 - val_average_precision: 0.4135 - val_AUCROC: 0.7891 - val_binary_accuracy: 0.8169
Epoch 195/200
7/7 [==============================] - 0s 31ms/step - loss: 0.3881 - average_precision: 0.4623 - AUCROC: 0.7995 - binary_accuracy: 0.8241 - val_loss: 0.3831 - val_average_precision: 0.4135 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.8169
Epoch 196/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3882 - average_precision: 0.4577 - AUCROC: 0.7995 - binary_accuracy: 0.8261 - val_loss: 0.3830 - val_average_precision: 0.4144 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.818

### Upsample Case

In [18]:
avg_precision_cross_val_up, roc_auc_cross_val_up, accuracy_cross_val_up = obtain_nn_scores(df_train, resampling = "Up")
print()
info(avg_precision_cross_val_up, roc_auc_cross_val_up, accuracy_cross_val_up)

Epoch 1/200
7/7 [==============================] - 0s 38ms/step - loss: 1.5629 - average_precision: 0.3020 - AUCROC: 0.6551 - binary_accuracy: 0.1803 - val_loss: 1.2963 - val_average_precision: 0.3612 - val_AUCROC: 0.7046 - val_binary_accuracy: 0.2013
Epoch 2/200
7/7 [==============================] - 0s 30ms/step - loss: 1.2009 - average_precision: 0.3504 - AUCROC: 0.7181 - binary_accuracy: 0.1936 - val_loss: 1.0151 - val_average_precision: 0.3795 - val_AUCROC: 0.7328 - val_binary_accuracy: 0.3119
Epoch 3/200
7/7 [==============================] - 0s 30ms/step - loss: 0.9398 - average_precision: 0.3504 - AUCROC: 0.7401 - binary_accuracy: 0.3786 - val_loss: 0.8099 - val_average_precision: 0.3951 - val_AUCROC: 0.7494 - val_binary_accuracy: 0.4819
Epoch 4/200
7/7 [==============================] - 0s 26ms/step - loss: 0.7502 - average_precision: 0.3759 - AUCROC: 0.7546 - binary_accuracy: 0.5334 - val_loss: 0.6629 - val_average_precision: 0.4166 - val_AUCROC: 0.7593 - val_binary_accuracy:

7/7 [==============================] - 0s 27ms/step - loss: 0.3822 - average_precision: 0.4528 - AUCROC: 0.7988 - binary_accuracy: 0.8244 - val_loss: 0.4113 - val_average_precision: 0.4660 - val_AUCROC: 0.7890 - val_binary_accuracy: 0.8087
Epoch 34/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3823 - average_precision: 0.4393 - AUCROC: 0.7985 - binary_accuracy: 0.8248 - val_loss: 0.4146 - val_average_precision: 0.4663 - val_AUCROC: 0.7899 - val_binary_accuracy: 0.8131
Epoch 35/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3821 - average_precision: 0.4564 - AUCROC: 0.7986 - binary_accuracy: 0.8255 - val_loss: 0.4111 - val_average_precision: 0.4661 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8087
Epoch 36/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3825 - average_precision: 0.4472 - AUCROC: 0.7977 - binary_accuracy: 0.8228 - val_loss: 0.4123 - val_average_precision: 0.4662 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8144
E

7/7 [==============================] - 0s 25ms/step - loss: 0.3818 - average_precision: 0.4315 - AUCROC: 0.7977 - binary_accuracy: 0.8241 - val_loss: 0.4127 - val_average_precision: 0.4662 - val_AUCROC: 0.7901 - val_binary_accuracy: 0.8100
Epoch 66/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3816 - average_precision: 0.4511 - AUCROC: 0.7991 - binary_accuracy: 0.8238 - val_loss: 0.4111 - val_average_precision: 0.4658 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.8156
Epoch 67/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3814 - average_precision: 0.4517 - AUCROC: 0.7987 - binary_accuracy: 0.8227 - val_loss: 0.4130 - val_average_precision: 0.4661 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8100
Epoch 68/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3814 - average_precision: 0.4679 - AUCROC: 0.8000 - binary_accuracy: 0.8252 - val_loss: 0.4104 - val_average_precision: 0.4664 - val_AUCROC: 0.7895 - val_binary_accuracy: 0.8069
E

7/7 [==============================] - 0s 27ms/step - loss: 0.3816 - average_precision: 0.4441 - AUCROC: 0.7979 - binary_accuracy: 0.8225 - val_loss: 0.4129 - val_average_precision: 0.4650 - val_AUCROC: 0.7898 - val_binary_accuracy: 0.8062
Epoch 98/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3813 - average_precision: 0.4606 - AUCROC: 0.7997 - binary_accuracy: 0.8242 - val_loss: 0.4102 - val_average_precision: 0.4654 - val_AUCROC: 0.7900 - val_binary_accuracy: 0.8119
Epoch 99/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3814 - average_precision: 0.4424 - AUCROC: 0.7989 - binary_accuracy: 0.8238 - val_loss: 0.4107 - val_average_precision: 0.4655 - val_AUCROC: 0.7904 - val_binary_accuracy: 0.8138
Epoch 100/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3815 - average_precision: 0.4575 - AUCROC: 0.7978 - binary_accuracy: 0.8230 - val_loss: 0.4101 - val_average_precision: 0.4655 - val_AUCROC: 0.7895 - val_binary_accuracy: 0.8119


7/7 [==============================] - 0s 30ms/step - loss: 0.3809 - average_precision: 0.4584 - AUCROC: 0.8003 - binary_accuracy: 0.8284 - val_loss: 0.4106 - val_average_precision: 0.4650 - val_AUCROC: 0.7896 - val_binary_accuracy: 0.8075
Epoch 130/200
7/7 [==============================] - 0s 30ms/step - loss: 0.3812 - average_precision: 0.4599 - AUCROC: 0.8001 - binary_accuracy: 0.8263 - val_loss: 0.4122 - val_average_precision: 0.4648 - val_AUCROC: 0.7902 - val_binary_accuracy: 0.8094
Epoch 131/200
7/7 [==============================] - 0s 29ms/step - loss: 0.3812 - average_precision: 0.4561 - AUCROC: 0.7996 - binary_accuracy: 0.8238 - val_loss: 0.4158 - val_average_precision: 0.4648 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.7994
Epoch 132/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3817 - average_precision: 0.4589 - AUCROC: 0.7989 - binary_accuracy: 0.8248 - val_loss: 0.4100 - val_average_precision: 0.4646 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.806

...

7/7 [==============================] - 0s 23ms/step - loss: 0.3913 - average_precision: 0.4502 - AUCROC: 0.7966 - binary_accuracy: 0.8197 - val_loss: 0.3885 - val_average_precision: 0.4076 - val_AUCROC: 0.7883 - val_binary_accuracy: 0.8144
Epoch 65/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3913 - average_precision: 0.4452 - AUCROC: 0.7968 - binary_accuracy: 0.8219 - val_loss: 0.3876 - val_average_precision: 0.4066 - val_AUCROC: 0.7882 - val_binary_accuracy: 0.8138
Epoch 66/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3911 - average_precision: 0.4445 - AUCROC: 0.7970 - binary_accuracy: 0.8202 - val_loss: 0.3875 - val_average_precision: 0.4066 - val_AUCROC: 0.7883 - val_binary_accuracy: 0.8125
Epoch 67/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3910 - average_precision: 0.4605 - AUCROC: 0.7969 - binary_accuracy: 0.8198 - val_loss: 0.3877 - val_average_precision: 0.4066 - val_AUCROC: 0.7888 - val_binary_accuracy: 0.8144
E

7/7 [==============================] - 0s 22ms/step - loss: 0.3890 - average_precision: 0.4583 - AUCROC: 0.7989 - binary_accuracy: 0.8228 - val_loss: 0.3859 - val_average_precision: 0.4126 - val_AUCROC: 0.7889 - val_binary_accuracy: 0.8156
Epoch 97/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3888 - average_precision: 0.4586 - AUCROC: 0.7992 - binary_accuracy: 0.8231 - val_loss: 0.3852 - val_average_precision: 0.4126 - val_AUCROC: 0.7889 - val_binary_accuracy: 0.8150
Epoch 98/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3889 - average_precision: 0.4639 - AUCROC: 0.7987 - binary_accuracy: 0.8228 - val_loss: 0.3855 - val_average_precision: 0.4126 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8156
Epoch 99/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3888 - average_precision: 0.4473 - AUCROC: 0.7987 - binary_accuracy: 0.8213 - val_loss: 0.3852 - val_average_precision: 0.4126 - val_AUCROC: 0.7888 - val_binary_accuracy: 0.8138
E

7/7 [==============================] - 0s 27ms/step - loss: 0.3883 - average_precision: 0.4490 - AUCROC: 0.7995 - binary_accuracy: 0.8241 - val_loss: 0.3847 - val_average_precision: 0.4138 - val_AUCROC: 0.7895 - val_binary_accuracy: 0.8150
Epoch 129/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3883 - average_precision: 0.4557 - AUCROC: 0.8000 - binary_accuracy: 0.8225 - val_loss: 0.3844 - val_average_precision: 0.4137 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8169
Epoch 130/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3883 - average_precision: 0.4711 - AUCROC: 0.7996 - binary_accuracy: 0.8238 - val_loss: 0.3849 - val_average_precision: 0.4137 - val_AUCROC: 0.7887 - val_binary_accuracy: 0.8169
Epoch 131/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3880 - average_precision: 0.4592 - AUCROC: 0.8001 - binary_accuracy: 0.8248 - val_loss: 0.3844 - val_average_precision: 0.4138 - val_AUCROC: 0.7891 - val_binary_accuracy: 0.815

7/7 [==============================] - 0s 26ms/step - loss: 0.3882 - average_precision: 0.4761 - AUCROC: 0.7996 - binary_accuracy: 0.8247 - val_loss: 0.3841 - val_average_precision: 0.4154 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8169
Epoch 161/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3879 - average_precision: 0.4622 - AUCROC: 0.8002 - binary_accuracy: 0.8242 - val_loss: 0.3839 - val_average_precision: 0.4154 - val_AUCROC: 0.7897 - val_binary_accuracy: 0.8150
Epoch 162/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3879 - average_precision: 0.4720 - AUCROC: 0.8002 - binary_accuracy: 0.8234 - val_loss: 0.3839 - val_average_precision: 0.4154 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8169
Epoch 163/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3881 - average_precision: 0.4760 - AUCROC: 0.7998 - binary_accuracy: 0.8231 - val_loss: 0.3843 - val_average_precision: 0.4153 - val_AUCROC: 0.7895 - val_binary_accuracy: 0.816

Epoch 192/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3881 - average_precision: 0.4712 - AUCROC: 0.7998 - binary_accuracy: 0.8255 - val_loss: 0.3840 - val_average_precision: 0.4131 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8169
Epoch 193/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3879 - average_precision: 0.4713 - AUCROC: 0.7996 - binary_accuracy: 0.8241 - val_loss: 0.3838 - val_average_precision: 0.4140 - val_AUCROC: 0.7888 - val_binary_accuracy: 0.8169
Epoch 194/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3881 - average_precision: 0.4746 - AUCROC: 0.7998 - binary_accuracy: 0.8247 - val_loss: 0.3839 - val_average_precision: 0.4130 - val_AUCROC: 0.7892 - val_binary_accuracy: 0.8169
Epoch 195/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3879 - average_precision: 0.4699 - AUCROC: 0.7999 - binary_accuracy: 0.8222 - val_loss: 0.3839 - val_average_precision: 0.4131 - val_AUCROC: 0.7886 - val_binary_a

### Downsample Case

In [19]:
avg_precision_cross_val_down, roc_auc_cross_val_down, accuracy_cross_val_down = obtain_nn_scores(df_train, resampling = "Down")
print()
info(avg_precision_cross_val_down, roc_auc_cross_val_down, accuracy_cross_val_down)

Epoch 1/200
7/7 [==============================] - 0s 32ms/step - loss: 0.4257 - average_precision: 0.4233 - AUCROC: 0.7846 - binary_accuracy: 0.8197 - val_loss: 0.4527 - val_average_precision: 0.4435 - val_AUCROC: 0.7731 - val_binary_accuracy: 0.7987
Epoch 2/200
7/7 [==============================] - 0s 24ms/step - loss: 0.4212 - average_precision: 0.4167 - AUCROC: 0.7864 - binary_accuracy: 0.8197 - val_loss: 0.4485 - val_average_precision: 0.4397 - val_AUCROC: 0.7725 - val_binary_accuracy: 0.7987
Epoch 3/200
7/7 [==============================] - 0s 25ms/step - loss: 0.4170 - average_precision: 0.4090 - AUCROC: 0.7840 - binary_accuracy: 0.8197 - val_loss: 0.4457 - val_average_precision: 0.4390 - val_AUCROC: 0.7708 - val_binary_accuracy: 0.7987
Epoch 4/200
7/7 [==============================] - 0s 37ms/step - loss: 0.4128 - average_precision: 0.4072 - AUCROC: 0.7839 - binary_accuracy: 0.8197 - val_loss: 0.4395 - val_average_precision: 0.4406 - val_AUCROC: 0.7701 - val_binary_accuracy:

7/7 [==============================] - 0s 29ms/step - loss: 0.3864 - average_precision: 0.4238 - AUCROC: 0.7946 - binary_accuracy: 0.8202 - val_loss: 0.4171 - val_average_precision: 0.4430 - val_AUCROC: 0.7835 - val_binary_accuracy: 0.8119
Epoch 34/200
7/7 [==============================] - 0s 29ms/step - loss: 0.3863 - average_precision: 0.4253 - AUCROC: 0.7950 - binary_accuracy: 0.8200 - val_loss: 0.4174 - val_average_precision: 0.4429 - val_AUCROC: 0.7832 - val_binary_accuracy: 0.8119
Epoch 35/200
7/7 [==============================] - 0s 32ms/step - loss: 0.3862 - average_precision: 0.4243 - AUCROC: 0.7950 - binary_accuracy: 0.8202 - val_loss: 0.4163 - val_average_precision: 0.4433 - val_AUCROC: 0.7835 - val_binary_accuracy: 0.8119
Epoch 36/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3858 - average_precision: 0.4150 - AUCROC: 0.7955 - binary_accuracy: 0.8202 - val_loss: 0.4162 - val_average_precision: 0.4433 - val_AUCROC: 0.7832 - val_binary_accuracy: 0.8119
E

7/7 [==============================] - 0s 23ms/step - loss: 0.3831 - average_precision: 0.4350 - AUCROC: 0.7977 - binary_accuracy: 0.8202 - val_loss: 0.4127 - val_average_precision: 0.4566 - val_AUCROC: 0.7877 - val_binary_accuracy: 0.8144
Epoch 66/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3833 - average_precision: 0.4533 - AUCROC: 0.7974 - binary_accuracy: 0.8213 - val_loss: 0.4127 - val_average_precision: 0.4566 - val_AUCROC: 0.7882 - val_binary_accuracy: 0.8144
Epoch 67/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3834 - average_precision: 0.4410 - AUCROC: 0.7975 - binary_accuracy: 0.8209 - val_loss: 0.4132 - val_average_precision: 0.4566 - val_AUCROC: 0.7884 - val_binary_accuracy: 0.8119
Epoch 68/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3830 - average_precision: 0.4323 - AUCROC: 0.7983 - binary_accuracy: 0.8205 - val_loss: 0.4129 - val_average_precision: 0.4566 - val_AUCROC: 0.7883 - val_binary_accuracy: 0.8119
E

Epoch 97/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3819 - average_precision: 0.4340 - AUCROC: 0.7987 - binary_accuracy: 0.8222 - val_loss: 0.4116 - val_average_precision: 0.4653 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8144
Epoch 98/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3824 - average_precision: 0.4444 - AUCROC: 0.7983 - binary_accuracy: 0.8220 - val_loss: 0.4114 - val_average_precision: 0.4653 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8138
Epoch 99/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3819 - average_precision: 0.4359 - AUCROC: 0.7989 - binary_accuracy: 0.8241 - val_loss: 0.4121 - val_average_precision: 0.4653 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8125
Epoch 100/200
7/7 [==============================] - 0s 26ms/step - loss: 0.3820 - average_precision: 0.4296 - AUCROC: 0.7987 - binary_accuracy: 0.8239 - val_loss: 0.4142 - val_average_precision: 0.4653 - val_AUCROC: 0.7898 - val_binary_accu

7/7 [==============================] - 0s 25ms/step - loss: 0.3820 - average_precision: 0.4622 - AUCROC: 0.7986 - binary_accuracy: 0.8228 - val_loss: 0.4107 - val_average_precision: 0.4658 - val_AUCROC: 0.7896 - val_binary_accuracy: 0.8106
Epoch 130/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3815 - average_precision: 0.4344 - AUCROC: 0.7996 - binary_accuracy: 0.8247 - val_loss: 0.4123 - val_average_precision: 0.4658 - val_AUCROC: 0.7891 - val_binary_accuracy: 0.8125
Epoch 131/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3816 - average_precision: 0.4508 - AUCROC: 0.7993 - binary_accuracy: 0.8203 - val_loss: 0.4110 - val_average_precision: 0.4658 - val_AUCROC: 0.7899 - val_binary_accuracy: 0.8125
Epoch 132/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3819 - average_precision: 0.4388 - AUCROC: 0.7987 - binary_accuracy: 0.8227 - val_loss: 0.4107 - val_average_precision: 0.4658 - val_AUCROC: 0.7897 - val_binary_accuracy: 0.815

...

Epoch 62/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3895 - average_precision: 0.4698 - AUCROC: 0.7991 - binary_accuracy: 0.8256 - val_loss: 0.3875 - val_average_precision: 0.4159 - val_AUCROC: 0.7885 - val_binary_accuracy: 0.8188
Epoch 63/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3893 - average_precision: 0.4637 - AUCROC: 0.7992 - binary_accuracy: 0.8252 - val_loss: 0.3869 - val_average_precision: 0.4159 - val_AUCROC: 0.7882 - val_binary_accuracy: 0.8163
Epoch 64/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3896 - average_precision: 0.4770 - AUCROC: 0.7992 - binary_accuracy: 0.8230 - val_loss: 0.3867 - val_average_precision: 0.4158 - val_AUCROC: 0.7885 - val_binary_accuracy: 0.8188
Epoch 65/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3895 - average_precision: 0.4614 - AUCROC: 0.7989 - binary_accuracy: 0.8252 - val_loss: 0.3865 - val_average_precision: 0.4159 - val_AUCROC: 0.7891 - val_binary_accur

7/7 [==============================] - 0s 25ms/step - loss: 0.3888 - average_precision: 0.4687 - AUCROC: 0.7995 - binary_accuracy: 0.8248 - val_loss: 0.3855 - val_average_precision: 0.4160 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8188
Epoch 95/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3888 - average_precision: 0.4643 - AUCROC: 0.7997 - binary_accuracy: 0.8252 - val_loss: 0.3856 - val_average_precision: 0.4163 - val_AUCROC: 0.7899 - val_binary_accuracy: 0.8188
Epoch 96/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3891 - average_precision: 0.4585 - AUCROC: 0.7988 - binary_accuracy: 0.8238 - val_loss: 0.3852 - val_average_precision: 0.4162 - val_AUCROC: 0.7895 - val_binary_accuracy: 0.8188
Epoch 97/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3888 - average_precision: 0.4682 - AUCROC: 0.7994 - binary_accuracy: 0.8252 - val_loss: 0.3864 - val_average_precision: 0.4163 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8175
E

7/7 [==============================] - 0s 22ms/step - loss: 0.3887 - average_precision: 0.4726 - AUCROC: 0.7992 - binary_accuracy: 0.8227 - val_loss: 0.3843 - val_average_precision: 0.4173 - val_AUCROC: 0.7899 - val_binary_accuracy: 0.8188
Epoch 127/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3886 - average_precision: 0.4677 - AUCROC: 0.7990 - binary_accuracy: 0.8256 - val_loss: 0.3843 - val_average_precision: 0.4173 - val_AUCROC: 0.7895 - val_binary_accuracy: 0.8188
Epoch 128/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3886 - average_precision: 0.4671 - AUCROC: 0.7990 - binary_accuracy: 0.8236 - val_loss: 0.3847 - val_average_precision: 0.4170 - val_AUCROC: 0.7899 - val_binary_accuracy: 0.8200
Epoch 129/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3884 - average_precision: 0.4578 - AUCROC: 0.7994 - binary_accuracy: 0.8245 - val_loss: 0.3848 - val_average_precision: 0.4172 - val_AUCROC: 0.7898 - val_binary_accuracy: 0.818

Epoch 158/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3883 - average_precision: 0.4680 - AUCROC: 0.7996 - binary_accuracy: 0.8244 - val_loss: 0.3838 - val_average_precision: 0.4180 - val_AUCROC: 0.7902 - val_binary_accuracy: 0.8188
Epoch 159/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3883 - average_precision: 0.4635 - AUCROC: 0.7993 - binary_accuracy: 0.8242 - val_loss: 0.3838 - val_average_precision: 0.4180 - val_AUCROC: 0.7902 - val_binary_accuracy: 0.8188
Epoch 160/200
7/7 [==============================] - 0s 27ms/step - loss: 0.3882 - average_precision: 0.4684 - AUCROC: 0.7996 - binary_accuracy: 0.8263 - val_loss: 0.3845 - val_average_precision: 0.4179 - val_AUCROC: 0.7904 - val_binary_accuracy: 0.8188
Epoch 161/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3884 - average_precision: 0.4723 - AUCROC: 0.7995 - binary_accuracy: 0.8248 - val_loss: 0.3840 - val_average_precision: 0.4193 - val_AUCROC: 0.7899 - val_binary_a

7/7 [==============================] - 0s 22ms/step - loss: 0.3883 - average_precision: 0.4694 - AUCROC: 0.7994 - binary_accuracy: 0.8245 - val_loss: 0.3837 - val_average_precision: 0.4191 - val_AUCROC: 0.7901 - val_binary_accuracy: 0.8188
Epoch 191/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3883 - average_precision: 0.4766 - AUCROC: 0.7995 - binary_accuracy: 0.8241 - val_loss: 0.3850 - val_average_precision: 0.4195 - val_AUCROC: 0.7904 - val_binary_accuracy: 0.8181
Epoch 192/200
7/7 [==============================] - 0s 20ms/step - loss: 0.3883 - average_precision: 0.4746 - AUCROC: 0.7993 - binary_accuracy: 0.8230 - val_loss: 0.3839 - val_average_precision: 0.4195 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.8188
Epoch 193/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3883 - average_precision: 0.4797 - AUCROC: 0.7990 - binary_accuracy: 0.8252 - val_loss: 0.3839 - val_average_precision: 0.4195 - val_AUCROC: 0.7905 - val_binary_accuracy: 0.818

### Balanced Sampling Case

In [20]:
avg_precision_cross_val_bal, roc_auc_cross_val_bal, accuracy_cross_val_bal = obtain_nn_scores(df_train, resampling = "Balanced")
print()
info(avg_precision_cross_val_bal, roc_auc_cross_val_bal, accuracy_cross_val_bal)

Epoch 1/200
7/7 [==============================] - 0s 29ms/step - loss: 0.4658 - average_precision: 0.2543 - AUCROC: 0.5976 - binary_accuracy: 0.8197 - val_loss: 0.4845 - val_average_precision: 0.3631 - val_AUCROC: 0.7080 - val_binary_accuracy: 0.7987
Epoch 2/200
7/7 [==============================] - 0s 21ms/step - loss: 0.4547 - average_precision: 0.3491 - AUCROC: 0.7034 - binary_accuracy: 0.8197 - val_loss: 0.4751 - val_average_precision: 0.4140 - val_AUCROC: 0.7535 - val_binary_accuracy: 0.7987
Epoch 3/200
7/7 [==============================] - 0s 24ms/step - loss: 0.4460 - average_precision: 0.3830 - AUCROC: 0.7457 - binary_accuracy: 0.8197 - val_loss: 0.4680 - val_average_precision: 0.4305 - val_AUCROC: 0.7691 - val_binary_accuracy: 0.7987
Epoch 4/200
7/7 [==============================] - 0s 23ms/step - loss: 0.4385 - average_precision: 0.4070 - AUCROC: 0.7624 - binary_accuracy: 0.8197 - val_loss: 0.4607 - val_average_precision: 0.4426 - val_AUCROC: 0.7761 - val_binary_accuracy:

7/7 [==============================] - 0s 25ms/step - loss: 0.3838 - average_precision: 0.4497 - AUCROC: 0.7969 - binary_accuracy: 0.8234 - val_loss: 0.4160 - val_average_precision: 0.4378 - val_AUCROC: 0.7872 - val_binary_accuracy: 0.7931
Epoch 34/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3836 - average_precision: 0.4364 - AUCROC: 0.7971 - binary_accuracy: 0.8214 - val_loss: 0.4133 - val_average_precision: 0.4378 - val_AUCROC: 0.7871 - val_binary_accuracy: 0.7962
Epoch 35/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3833 - average_precision: 0.4327 - AUCROC: 0.7969 - binary_accuracy: 0.8223 - val_loss: 0.4153 - val_average_precision: 0.4375 - val_AUCROC: 0.7872 - val_binary_accuracy: 0.7962
Epoch 36/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3833 - average_precision: 0.4454 - AUCROC: 0.7974 - binary_accuracy: 0.8230 - val_loss: 0.4130 - val_average_precision: 0.4382 - val_AUCROC: 0.7872 - val_binary_accuracy: 0.7962
E

7/7 [==============================] - 0s 20ms/step - loss: 0.3818 - average_precision: 0.4371 - AUCROC: 0.7985 - binary_accuracy: 0.8256 - val_loss: 0.4142 - val_average_precision: 0.4514 - val_AUCROC: 0.7875 - val_binary_accuracy: 0.8025
Epoch 66/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3820 - average_precision: 0.4460 - AUCROC: 0.7981 - binary_accuracy: 0.8264 - val_loss: 0.4145 - val_average_precision: 0.4515 - val_AUCROC: 0.7882 - val_binary_accuracy: 0.8025
Epoch 67/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3820 - average_precision: 0.4473 - AUCROC: 0.7978 - binary_accuracy: 0.8241 - val_loss: 0.4163 - val_average_precision: 0.4516 - val_AUCROC: 0.7882 - val_binary_accuracy: 0.7994
Epoch 68/200
7/7 [==============================] - 0s 20ms/step - loss: 0.3825 - average_precision: 0.4486 - AUCROC: 0.7968 - binary_accuracy: 0.8233 - val_loss: 0.4126 - val_average_precision: 0.4521 - val_AUCROC: 0.7882 - val_binary_accuracy: 0.8012
E

7/7 [==============================] - 0s 23ms/step - loss: 0.3816 - average_precision: 0.4587 - AUCROC: 0.7982 - binary_accuracy: 0.8263 - val_loss: 0.4106 - val_average_precision: 0.4518 - val_AUCROC: 0.7875 - val_binary_accuracy: 0.8075
Epoch 98/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3818 - average_precision: 0.4623 - AUCROC: 0.7984 - binary_accuracy: 0.8242 - val_loss: 0.4105 - val_average_precision: 0.4517 - val_AUCROC: 0.7876 - val_binary_accuracy: 0.8094
Epoch 99/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3815 - average_precision: 0.4603 - AUCROC: 0.7985 - binary_accuracy: 0.8248 - val_loss: 0.4105 - val_average_precision: 0.4522 - val_AUCROC: 0.7881 - val_binary_accuracy: 0.8094
Epoch 100/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3814 - average_precision: 0.4614 - AUCROC: 0.7987 - binary_accuracy: 0.8242 - val_loss: 0.4105 - val_average_precision: 0.4522 - val_AUCROC: 0.7880 - val_binary_accuracy: 0.8062


Epoch 129/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3814 - average_precision: 0.4469 - AUCROC: 0.7986 - binary_accuracy: 0.8241 - val_loss: 0.4105 - val_average_precision: 0.4586 - val_AUCROC: 0.7887 - val_binary_accuracy: 0.8094
Epoch 130/200
7/7 [==============================] - 0s 20ms/step - loss: 0.3818 - average_precision: 0.4613 - AUCROC: 0.7976 - binary_accuracy: 0.8241 - val_loss: 0.4104 - val_average_precision: 0.4586 - val_AUCROC: 0.7882 - val_binary_accuracy: 0.8075
Epoch 131/200
7/7 [==============================] - 0s 25ms/step - loss: 0.3821 - average_precision: 0.4534 - AUCROC: 0.7970 - binary_accuracy: 0.8230 - val_loss: 0.4103 - val_average_precision: 0.4587 - val_AUCROC: 0.7890 - val_binary_accuracy: 0.8056
Epoch 132/200
7/7 [==============================] - 0s 24ms/step - loss: 0.3814 - average_precision: 0.4507 - AUCROC: 0.7991 - binary_accuracy: 0.8241 - val_loss: 0.4112 - val_average_precision: 0.4584 - val_AUCROC: 0.7888 - val_binary_a

...

7/7 [==============================] - 0s 22ms/step - loss: 0.3894 - average_precision: 0.4822 - AUCROC: 0.7989 - binary_accuracy: 0.8217 - val_loss: 0.3874 - val_average_precision: 0.4147 - val_AUCROC: 0.7893 - val_binary_accuracy: 0.8138
Epoch 65/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3896 - average_precision: 0.4577 - AUCROC: 0.7988 - binary_accuracy: 0.8236 - val_loss: 0.3861 - val_average_precision: 0.4119 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8188
Epoch 66/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3894 - average_precision: 0.4617 - AUCROC: 0.7988 - binary_accuracy: 0.8253 - val_loss: 0.3859 - val_average_precision: 0.4119 - val_AUCROC: 0.7888 - val_binary_accuracy: 0.8163
Epoch 67/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3895 - average_precision: 0.4591 - AUCROC: 0.7988 - binary_accuracy: 0.8245 - val_loss: 0.3860 - val_average_precision: 0.4121 - val_AUCROC: 0.7892 - val_binary_accuracy: 0.8150
E

Epoch 96/200
7/7 [==============================] - 0s 20ms/step - loss: 0.3888 - average_precision: 0.4565 - AUCROC: 0.7991 - binary_accuracy: 0.8239 - val_loss: 0.3848 - val_average_precision: 0.4125 - val_AUCROC: 0.7891 - val_binary_accuracy: 0.8156
Epoch 97/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3886 - average_precision: 0.4583 - AUCROC: 0.7993 - binary_accuracy: 0.8241 - val_loss: 0.3847 - val_average_precision: 0.4125 - val_AUCROC: 0.7891 - val_binary_accuracy: 0.8169
Epoch 98/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3884 - average_precision: 0.4506 - AUCROC: 0.7994 - binary_accuracy: 0.8247 - val_loss: 0.3862 - val_average_precision: 0.4122 - val_AUCROC: 0.7892 - val_binary_accuracy: 0.8194
Epoch 99/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3891 - average_precision: 0.4610 - AUCROC: 0.7986 - binary_accuracy: 0.8222 - val_loss: 0.3849 - val_average_precision: 0.4125 - val_AUCROC: 0.7890 - val_binary_accur

7/7 [==============================] - 0s 25ms/step - loss: 0.3886 - average_precision: 0.4579 - AUCROC: 0.7990 - binary_accuracy: 0.8252 - val_loss: 0.3842 - val_average_precision: 0.4123 - val_AUCROC: 0.7888 - val_binary_accuracy: 0.8169
Epoch 129/200
7/7 [==============================] - 0s 23ms/step - loss: 0.3884 - average_precision: 0.4622 - AUCROC: 0.7993 - binary_accuracy: 0.8241 - val_loss: 0.3844 - val_average_precision: 0.4123 - val_AUCROC: 0.7885 - val_binary_accuracy: 0.8138
Epoch 130/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3883 - average_precision: 0.4699 - AUCROC: 0.7994 - binary_accuracy: 0.8253 - val_loss: 0.3842 - val_average_precision: 0.4123 - val_AUCROC: 0.7881 - val_binary_accuracy: 0.8163
Epoch 131/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3884 - average_precision: 0.4535 - AUCROC: 0.7999 - binary_accuracy: 0.8255 - val_loss: 0.3847 - val_average_precision: 0.4123 - val_AUCROC: 0.7891 - val_binary_accuracy: 0.817

7/7 [==============================] - 0s 22ms/step - loss: 0.3884 - average_precision: 0.4704 - AUCROC: 0.7993 - binary_accuracy: 0.8236 - val_loss: 0.3837 - val_average_precision: 0.4126 - val_AUCROC: 0.7891 - val_binary_accuracy: 0.8169
Epoch 161/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3883 - average_precision: 0.4635 - AUCROC: 0.7995 - binary_accuracy: 0.8259 - val_loss: 0.3837 - val_average_precision: 0.4126 - val_AUCROC: 0.7897 - val_binary_accuracy: 0.8156
Epoch 162/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3885 - average_precision: 0.4715 - AUCROC: 0.7991 - binary_accuracy: 0.8261 - val_loss: 0.3837 - val_average_precision: 0.4126 - val_AUCROC: 0.7898 - val_binary_accuracy: 0.8169
Epoch 163/200
7/7 [==============================] - 0s 22ms/step - loss: 0.3884 - average_precision: 0.4582 - AUCROC: 0.7991 - binary_accuracy: 0.8256 - val_loss: 0.3837 - val_average_precision: 0.4131 - val_AUCROC: 0.7895 - val_binary_accuracy: 0.816

7/7 [==============================] - 0s 20ms/step - loss: 0.3886 - average_precision: 0.4731 - AUCROC: 0.7991 - binary_accuracy: 0.8247 - val_loss: 0.3836 - val_average_precision: 0.4154 - val_AUCROC: 0.7890 - val_binary_accuracy: 0.8169
Epoch 192/200
7/7 [==============================] - 0s 20ms/step - loss: 0.3882 - average_precision: 0.4674 - AUCROC: 0.7999 - binary_accuracy: 0.8233 - val_loss: 0.3835 - val_average_precision: 0.4154 - val_AUCROC: 0.7892 - val_binary_accuracy: 0.8169
Epoch 193/200
7/7 [==============================] - 0s 21ms/step - loss: 0.3883 - average_precision: 0.4614 - AUCROC: 0.7996 - binary_accuracy: 0.8239 - val_loss: 0.3835 - val_average_precision: 0.4154 - val_AUCROC: 0.7894 - val_binary_accuracy: 0.8169
Epoch 194/200
7/7 [==============================] - ETA: 0s - loss: 0.3885 - average_precision: 0.4783 - AUCROC: 0.7992 - binary_accuracy: 0.82 - 0s 21ms/step - loss: 0.3885 - average_precision: 0.4783 - AUCROC: 0.7992 - binary_accuracy: 0.8233 - val_